# Imports

In [ ]:
import glob
import os
import random
import pathlib

import pandas as pd
import torch
from Bio.PDB.Polypeptide import index_to_one
from collections import OrderedDict
from torch.utils.data import DataLoader, Dataset

from cavity_model import (
    CavityModel,
    ResidueEnvironment,
    ResidueEnvironmentsDataset,
)
from helpers import (
    _augment_with_reverse_mutation,
    _populate_dfs_with_nlls_and_nlfs,
    _populate_dfs_with_resenvs,
    _train_loop,
    _train_val_split,
    _get_ddg_training_dataloaders,
    _get_ddg_validation_dataloaders,
    _train_downstream_and_evaluate,
)
from visualization import scatter_pred_vs_true, plot_validation_performance

%load_ext nb_black

# Cavity Model

Download and process Cavity Model data

In [ ]:
# Hack to find the conda activate path since bash scripts 
# don't necessarily work with the conda activate command
conda_path = !which conda
conda_path = list(conda_path)[0]
conda_activate_path = pathlib.Path(conda_path).parent.parent / "bin" / "activate"
if not conda_activate_path.is_file():
    raise FileNotFoundError(
        "Could not find your conda activate path needed for running bash scripts."
    )

In [ ]:
# Run shell script that takes a .txt file with PDBIDs as input.
!./get_and_parse_pdbs_for_cavity_model.sh $conda_activate_path data/pdbids_2336.txt

Global variables

In [ ]:
# Main parameters
DEVICE = "cuda"  # "cpu" or "cuda"
TRAIN_VAL_SPLIT = 0.8
BATCH_SIZE = 100
SHUFFLE_PDBS = True
LEARNING_RATE = 3e-4
EPOCHS = 10
PATIENCE_CUTOFF = 1
EPS = 1e-9

# Parameters specific to downstream model
BATCH_SIZE_DDG = 40
SHUFFLE_DDG = True
LEARNING_RATE_DDG = 1e-3
EPOCHS_DDG = 200

Load Parsed PDBs and perform train/val split

In [ ]:
parsed_pdb_filenames = sorted(glob.glob("data/pdbs/parsed/*coord*"))
if SHUFFLE_PDBS:
    random.shuffle(parsed_pdb_filenames)
dataloader_train, dataset_train, dataloader_val, dataset_val = _train_val_split(
    parsed_pdb_filenames, TRAIN_VAL_SPLIT, DEVICE, BATCH_SIZE
)

Train the cavity model

In [ ]:
# Define model
cavity_model_net = CavityModel(DEVICE).to(DEVICE)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cavity_model_net.parameters(), lr=LEARNING_RATE)

# Create directory for model files
models_dirpath = "cavity_models/"
if not os.path.exists(models_dirpath):
    os.mkdir(models_dirpath)

# Train loop
best_model_path = _train_loop(
    dataloader_train,
    dataloader_val,
    cavity_model_net,
    loss_function,
    optimizer,
    EPOCHS,
    PATIENCE_CUTOFF,
)

# ddG Prediction

Parse PDBs for DMS, Guerois and Protein G data sets

In [ ]:
# Parse PDBs for which we have ddG data
!./get_and_parse_pdbs_for_dowstream_task.sh $conda_activate_path

Make dict for residue environments for easy look up

In [ ]:
# Create temporary residue environment datasets as dicts to more easily match ddG data
parsed_pdbs_wildcards = {
    "dms": "data/data_dms/pdbs_parsed/*coord*",
    "protein_g": "data/data_protein_g/pdbs_parsed/*coord*",
    "guerois": "data/data_guerois/pdbs_parsed/*coord*",
    "symmetric": "data/data_symmetric/pdbs_parsed/*coord*",
}

resenv_datasets_look_up = {}
for dataset_key, pdbs_wildcard in parsed_pdbs_wildcards.items():
    parsed_pdb_filenames = sorted(glob.glob(pdbs_wildcard))
    dataset = ResidueEnvironmentsDataset(parsed_pdb_filenames, transformer=None)
    dataset_look_up = {}
    for resenv in dataset:
        key = (
            f"{resenv.pdb_id}{resenv.chain_id}_{resenv.pdb_residue_number}"
            f"{index_to_one(resenv.restype_index)}"
        )
        dataset_look_up[key] = resenv
    resenv_datasets_look_up[dataset_key] = dataset_look_up

Load ddG data to dataframe

In [ ]:
ddg_data_dict = OrderedDict()
ddg_data_dict = {
    "dms": pd.read_csv("data/data_dms/ddgs_parsed.csv"),
    "protein_g": pd.read_csv("data/data_protein_g/ddgs_parsed.csv"),
    "guerois": pd.read_csv("data/data_guerois/ddgs_parsed.csv"),
    "symmetric_direct": pd.read_csv("data/data_symmetric/ddgs_parsed_direct.csv"),
    "symmetric_inverse": pd.read_csv("data/data_symmetric/ddgs_parsed_inverse.csv"),
}

Populate dataframes with wt ResidueEnvironment objects and wt and mt restype indices

In [ ]:
_populate_dfs_with_resenvs(ddg_data_dict, resenv_datasets_look_up)

Populate dataframes with predicted NLLs and isolated WT and MT predicted NLLs as well as NLFs

In [ ]:
# Load best performing cavity model from previos training
cavity_model_infer_net = CavityModel(DEVICE).to(DEVICE)
cavity_model_infer_net.load_state_dict(torch.load(best_model_path))
cavity_model_infer_net.eval()

_populate_dfs_with_nlls_and_nlfs(
    ddg_data_dict, cavity_model_infer_net, DEVICE, BATCH_SIZE, EPS
)

### Results without downstream model

In [ ]:
colors = ["steelblue", "firebrick", "forestgreen", "black", "black"]
for color, data_key in zip(colors, ddg_data_dict.keys()):
    fig, ax = scatter_pred_vs_true(
        ddg_data_dict[data_key]["ddg"],
        ddg_data_dict[data_key]["ddg_pred_no_ds"],
        color=color,
        title=data_key,
    )

Merge direct and inverse dfs

In [ ]:
# Rename columns so they specify if it is the direct or inverse direction
symmetric_direct_df = ddg_data_dict["symmetric_direct"]
symmetric_direct_df.columns = [
    name + "_dir" if "_dir" not in name else name
    for name in symmetric_direct_df.columns
]
symmetric_inverse_df = ddg_data_dict["symmetric_inverse"]
symmetric_inverse_df.columns = [
    name + "_inv" if "_inv" not in name else name
    for name in symmetric_inverse_df.columns
]

# Inner merge both dataframes
ddg_data_dict["symmetric_both"] = pd.merge(
    symmetric_direct_df,
    symmetric_inverse_df,
    how="inner",
    left_on="merge_column_dir",
    right_on="merge_column_inv",
)

Make prediction based on both structures

In [ ]:
symmetric_both_df = ddg_data_dict["symmetric_both"]
symmetric_both_df["ddg_pred_no_ds_both_dir"] = symmetric_both_df.apply(
    lambda row: 0.5 * (row["ddg_pred_no_ds_dir"] - row["ddg_pred_no_ds_inv"]), axis=1
)
symmetric_both_df["ddg_pred_no_ds_both_inv"] = symmetric_both_df.apply(
    lambda row: 0.5 * (row["ddg_pred_no_ds_inv"] - row["ddg_pred_no_ds_dir"]), axis=1
)

Plot prediction using both structures

In [ ]:
# Plot prediction for ddg direct using both structures
fig, ax = scatter_pred_vs_true(
    ddg_data_dict["symmetric_both"]["ddg_dir"],
    ddg_data_dict["symmetric_both"]["ddg_pred_no_ds_both_dir"],
    color="black",
    title="Both structure (Direct)",
)

# Plot prediction for ddg inverse using both structures
fig, ax = scatter_pred_vs_true(
    ddg_data_dict["symmetric_both"]["ddg_inv"],
    ddg_data_dict["symmetric_both"]["ddg_pred_no_ds_both_inv"],
    color="black",
    title="Both structure (Inverse)",
)

# Downstream model

### Performance without augmentation

Define training dataloader and eval dataloaders

In [ ]:
ddg_dataloaders_train_dict = _get_ddg_training_dataloaders(
    ddg_data_dict, BATCH_SIZE_DDG, SHUFFLE_DDG
)
ddg_dataloaders_val_dict = _get_ddg_validation_dataloaders(ddg_data_dict)

Train and report on the data that is not used during training

In [ ]:
pearsons_r_results_dict = _train_downstream_and_evaluate(
    ddg_dataloaders_train_dict,
    ddg_dataloaders_val_dict,
    DEVICE,
    LEARNING_RATE_DDG,
    EPOCHS_DDG,
)

In [ ]:
for key in pearsons_r_results_dict.keys():
    _ = plot_validation_performance(key, pearsons_r_results_dict[key])

### Performance with augmentation

Before training we "augment" our dataset simply by adding the reverse mutation with -ddG value

In [ ]:
ddg_data_dict_augmented = _augment_with_reverse_mutation(ddg_data_dict)

Define training dataloader (augmented data) and eval dataloaders (original data)

In [ ]:
ddg_dataloaders_train_dict = _get_ddg_training_dataloaders(
    ddg_data_dict_augmented, BATCH_SIZE_DDG, SHUFFLE_DDG
)
ddg_dataloaders_val_dict = _get_ddg_validation_dataloaders(ddg_data_dict)

Train and report on the data that is not used during training

In [ ]:
pearsons_r_results_dict = _train_downstream_and_evaluate(
    ddg_dataloaders_train_dict,
    ddg_dataloaders_val_dict,
    DEVICE,
    LEARNING_RATE_DDG,
    EPOCHS_DDG,
)

In [ ]:
for key in pearsons_r_results_dict.keys():
    _ = plot_validation_performance(key, pearsons_r_results_dict[key])